import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

load dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sentiment_analysis_project/dataset/app_reviews.csv')

In [4]:
#Tidak di clean karena sudah di clean saat scraping dari Google playstore
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,5423592c-020f-4938-9f04-73217847417c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Fitur gofood nya kyk sampah, dikasih opsi peng...",1,62,5.32.1,2025-10-18 07:16:22,"Hai Kak Rizi, mohon maaf atas ketidaknyamanann...",2025-10-18 09:56:11,5.32.1
1,8e1753e4-43c1-4d7a-91ac-37efffcd5cf2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat jelek.. tidak bisa chat / telp resto ny...,1,18,5.35.1,2025-10-24 05:30:52,"Hai Kak Novia, mohon maaf atas ketidaknyamanan...",2025-10-24 06:06:31,5.35.1
2,ee4d3ed4-6f95-4cb4-ac39-457730b5b8bd,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,PELAYANAN BURUK! sudah cancel orderan tapi sal...,1,25,5.35.1,2025-10-19 01:30:28,"Hai Kak Jesica, mohon maat atas ketidaknyamana...",2025-10-19 06:40:41,5.35.1
3,a3ae98ae-6e99-49e9-a2c3-ce9a6dfa2f9e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Setiap kali pesan Gojek, data dan GPS saya akt...",1,48,5.35.1,2025-10-19 11:51:44,"Hai Kak Nabil, mohon maaf atas ketidaknyamanan...",2025-10-19 12:06:23,5.35.1
4,f2d90c51-3f25-4614-bde4-3960f50cd929,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong beri perlindungan untuk nasabah yang pe...,1,33,5.36.2,2025-10-24 10:55:00,"Hai Kak Putu, mohon maaf atas ketidaknyamanann...",2025-10-24 11:28:51,5.36.2


preprocessing teks

In [5]:
!pip install Sastrawi

In [6]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')
    return text

def casefoldingText(text):
    text = text.lower()
    return text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmingText(text):
    return stemmer.stem(text)

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [12]:
slangwords = {
    # Umum
    "@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual",
    "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "sdh": "sudah", "kyk": "seperti",

    # Kata kerja informal
    "gak": "tidak", "ga": "tidak", "gk": "tidak", "ngga": "tidak", "nggak": "tidak",
    "udh": "sudah", "udah": "sudah", "dah": "sudah", "blm": "belum", "blom": "belum",
    "pengen": "ingin", "pgn": "ingin", "mau": "ingin", "mo": "mau",
    "bisa": "dapat", "bs": "bisa", "jd": "jadi", "jadi": "menjadi",

    # Kata sifat informal
    "jelek": "buruk", "keren": "bagus", "mantap": "bagus", "mantul": "bagus",
    "anjlok": "turun", "naik": "meningkat", "lemot": "lambat", "lelet": "lambat",
    "cepet": "cepat", "cpt": "cepat", "lambat": "pelan",

    # Kata ganti dan pronomina
    "gue": "saya", "gw": "saya", "ane": "saya", "aku": "saya",
    "lu": "kamu", "loe": "kamu", "elu": "kamu", "lo": "kamu",
    "dia": "mereka", "mrk": "mereka", "kalian": "kamu",

    # Kata keterangan
    "aja": "saja", "aj": "saja", "doang": "saja", "dong": "saja",
    "banget": "sangat", "bgt": "sangat", "bener": "benar", "bnr": "benar",
    "emang": "memang", "emg": "memang", "sih": "memang",

    # Aplikasi spesifik
    "apps": "aplikasi", "app": "aplikasi", "apk": "aplikasi",
    "gopay": "pembayaran", "gofood": "makanan", "goride": "transportasi",
    "gocar": "transportasi", "gosend": "pengiriman",

    # Kata hubung dan partikel
    "krn": "karena", "krna": "karena", "soalnya": "karena", "soale": "karena",
    "tp": "tetapi", "tapi": "tetapi", "tp": "tetapi", "tpi": "tetapi",
    "sm": "sama", "dgn": "dengan", "dg": "dengan", "ama": "sama",

    # Ekspresi dan emotif
    "wkwk": "haha", "wkwkwk": "haha", "kwkw": "haha", "hehe": "haha",
    "huhu": "sedih", "hikz": "sedih", "hadeh": "kecewa", "aduh": "kecewa",
    "wow": "bagus", "wah": "bagus", "duh": "kecewa",

    # Typo umum
    "trima": "terima", "mksh": "terima kasih", "thx": "terima kasih", "tq": "terima kasih",
    "plz": "tolong", "pls": "tolong", "tlg": "tolong", "tolong": "mohon",
    "sgt": "sangat", "bner": "benar", "bnyk": "banyak", "sdikit": "sedikit",

    # Angka dan waktu
    "kmrn": "kemarin", "hr": "hari", "bsk": "besok", "thn": "tahun",
    "bln": "bulan", "mgg": "minggu", "jm": "jam", "mnt": "menit",

    # Bisnis/transaksi
    "dpt": "dapat", "gratis": "bebas", "disc": "diskon", "cashback": "kembalian",
    "voucher": "kupon", "promo": "promosi", "refund": "pengembalian",

    # Teknis
    "bug": "error", "lag": "lambat", "loading": "memuat", "crash": "rusak",
    "update": "pembaruan", "upgrade": "peningkatan", "downgrade": "penurunan",
    "install": "pasang", "uninstall": "hapus", "login": "masuk", "logout": "keluar"
}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [18]:
# Menerapkan preprocessing text
df['text_clean'] = df['content'].apply(cleaningText)

df['text_casefoldingText'] = df['text_clean'].apply(casefoldingText)

df['text_stemming'] = df['text_casefoldingText'].apply(stemmingText)

df['text_slangwords'] = df['text_stemming'].apply(fix_slangwords)

df['text_tokenizingText'] = df['text_slangwords'].apply(tokenizingText)

df['text_stopword'] = df['text_tokenizingText'].apply(filteringText)

df['text_akhir'] = df['text_stopword'].apply(toSentence)

In [19]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_stemming,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,5423592c-020f-4938-9f04-73217847417c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Fitur gofood nya kyk sampah, dikasih opsi peng...",1,62,5.32.1,2025-10-18 07:16:22,"Hai Kak Rizi, mohon maaf atas ketidaknyamanann...",2025-10-18 09:56:11,5.32.1,Fitur gofood nya kyk sampah dikasih opsi penga...,fitur gofood nya kyk sampah dikasih opsi penga...,fitur gofood nya kyk sampah kasih opsi antar c...,fitur makanan nya seperti sampah kasih opsi an...,"[fitur, makanan, nya, seperti, sampah, kasih, ...","[fitur, makanan, sampah, kasih, opsi, cepat, s...",fitur makanan sampah kasih opsi cepat sedeng p...
1,8e1753e4-43c1-4d7a-91ac-37efffcd5cf2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat jelek.. tidak bisa chat / telp resto ny...,1,18,5.35.1,2025-10-24 05:30:52,"Hai Kak Novia, mohon maaf atas ketidaknyamanan...",2025-10-24 06:06:31,5.35.1,sangat jelek tidak bisa chat telp resto nya h...,sangat jelek tidak bisa chat telp resto nya h...,sangat jelek tidak bisa chat telp resto nya ha...,sangat buruk tidak dapat chat telp resto nya h...,"[sangat, buruk, tidak, dapat, chat, telp, rest...","[buruk, chat, telp, resto, ganti, x, driver, r...",buruk chat telp resto ganti x driver resto tem...
2,ee4d3ed4-6f95-4cb4-ac39-457730b5b8bd,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,PELAYANAN BURUK! sudah cancel orderan tapi sal...,1,25,5.35.1,2025-10-19 01:30:28,"Hai Kak Jesica, mohon maat atas ketidaknyamana...",2025-10-19 06:40:41,5.35.1,PELAYANAN BURUK sudah cancel orderan tapi sald...,pelayanan buruk sudah cancel orderan tapi sald...,layan buruk sudah cancel order tapi saldo ga b...,layan buruk sudah cancel order tetapi saldo ti...,"[layan, buruk, sudah, cancel, order, tetapi, s...","[layan, buruk, cancel, order, saldo, wth, kerj...",layan buruk cancel order saldo wth kerja gaada...
3,a3ae98ae-6e99-49e9-a2c3-ce9a6dfa2f9e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Setiap kali pesan Gojek, data dan GPS saya akt...",1,48,5.35.1,2025-10-19 11:51:44,"Hai Kak Nabil, mohon maaf atas ketidaknyamanan...",2025-10-19 12:06:23,5.35.1,Setiap kali pesan Gojek data dan GPS saya akti...,setiap kali pesan gojek data dan gps saya akti...,tiap kali pesan gojek data dan gps saya aktif ...,tiap kali pesan gojek data dan gps saya aktif ...,"[tiap, kali, pesan, gojek, data, dan, gps, say...","[kali, pesan, gojek, data, gps, aktif, tekan, ...",kali pesan gojek data gps aktif tekan booking ...
4,f2d90c51-3f25-4614-bde4-3960f50cd929,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong beri perlindungan untuk nasabah yang pe...,1,33,5.36.2,2025-10-24 10:55:00,"Hai Kak Putu, mohon maaf atas ketidaknyamanann...",2025-10-24 11:28:51,5.36.2,Tolong beri perlindungan untuk nasabah yang pe...,tolong beri perlindungan untuk nasabah yang pe...,tolong beri lindung untuk nasabah yang pesan g...,mohon beri lindung untuk nasabah yang pesan tr...,"[mohon, beri, lindung, untuk, nasabah, yang, p...","[mohon, lindung, nasabah, pesan, transportasi,...",mohon lindung nasabah pesan transportasi trans...


Pelabelan

Lexicon

In [67]:
lexicon_indonesia = {
    "positive": [
        "bagus", "mantap", "cepat", "murah", "praktis", "puas", "keren",
        "hebat", "terbaik", "nyaman", "lancar", "mudah", "responsif",
        "ramah", "aman", "bersih", "teratur", "efisien", "recommended",
        "suka", "senang", "memuaskan", "worth it", "berguna", "bantu",
        "tepat waktu", "update bagus", "stabil", "fitur lengkap"
    ],

    "negative": [
        "buruk", "jelek", "lemot", "lambat", "gagal", "error", "macet",
        "susah", "ribet", "ngehang", "parah", "bohong", "kecewa",
        "bug", "jelek banget", "sampah", "tidak bisa", "tidak jalan",
        "ngelag", "crash", "lag", "tidak respon", "bikin kesel",
        "mengecewakan", "bermasalah", "update buruk", "iklan banyak",
        "fitur hilang", "data hilang", "akun terblokir"
    ],

    "neutral": [
        "biasa", "cukup", "lumayan", "oke", "so so", "standar",
        "seadanya", "bisa digunakan", "ya begitulah", "tidak buruk",
        "normal", "perlu diperbaiki", "harap diperbaiki", "butuh update",
        "belum dicoba", "masih tes", "belum tahu", "belum jelas"
    ]
}

In [63]:
def sentiment_analysis_lexicon_indonesia(text, lexicon):
    score = 0

    for word in text:
        if word in lexicon['positive']:
            score += 1
    for word in text:
        if word in lexicon['negative']:
            score -= 1

    # Penentuan polaritas
    if score > 0:
        polarity = 'positive'
    elif score == 0:
        polarity = 'neutral'
    else:
        polarity = 'negative'

    return score, polarity

In [68]:
results = df['text_stopword'].apply(lambda x: sentiment_analysis_lexicon_indonesia(x, lexicon_indonesia))
results = list(zip(*results))
df['polarity_score'] = results[0]
df['polarity'] = results[1]
print(df['polarity'].value_counts())

polarity
neutral     21940
negative     8640
positive     5202
Name: count, dtype: int64


In [83]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,...,text_casefoldingText,text_stemming,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity,label,sentiment
0,5423592c-020f-4938-9f04-73217847417c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Fitur gofood nya kyk sampah, dikasih opsi peng...",1,62,5.32.1,2025-10-18 07:16:22,"Hai Kak Rizi, mohon maaf atas ketidaknyamanann...",2025-10-18 09:56:11,...,fitur gofood nya kyk sampah dikasih opsi penga...,fitur gofood nya kyk sampah kasih opsi antar c...,fitur makanan nya seperti sampah kasih opsi an...,"[fitur, makanan, nya, seperti, sampah, kasih, ...","[fitur, makanan, sampah, kasih, opsi, cepat, s...",fitur makanan sampah kasih opsi cepat sedeng p...,0,neutral,1,negatif
1,8e1753e4-43c1-4d7a-91ac-37efffcd5cf2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat jelek.. tidak bisa chat / telp resto ny...,1,18,5.35.1,2025-10-24 05:30:52,"Hai Kak Novia, mohon maaf atas ketidaknyamanan...",2025-10-24 06:06:31,...,sangat jelek tidak bisa chat telp resto nya h...,sangat jelek tidak bisa chat telp resto nya ha...,sangat buruk tidak dapat chat telp resto nya h...,"[sangat, buruk, tidak, dapat, chat, telp, rest...","[buruk, chat, telp, resto, ganti, x, driver, r...",buruk chat telp resto ganti x driver resto tem...,-1,negative,0,negatif
2,ee4d3ed4-6f95-4cb4-ac39-457730b5b8bd,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,PELAYANAN BURUK! sudah cancel orderan tapi sal...,1,25,5.35.1,2025-10-19 01:30:28,"Hai Kak Jesica, mohon maat atas ketidaknyamana...",2025-10-19 06:40:41,...,pelayanan buruk sudah cancel orderan tapi sald...,layan buruk sudah cancel order tapi saldo ga b...,layan buruk sudah cancel order tetapi saldo ti...,"[layan, buruk, sudah, cancel, order, tetapi, s...","[layan, buruk, cancel, order, saldo, wth, kerj...",layan buruk cancel order saldo wth kerja gaada...,-1,negative,0,negatif
3,a3ae98ae-6e99-49e9-a2c3-ce9a6dfa2f9e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Setiap kali pesan Gojek, data dan GPS saya akt...",1,48,5.35.1,2025-10-19 11:51:44,"Hai Kak Nabil, mohon maaf atas ketidaknyamanan...",2025-10-19 12:06:23,...,setiap kali pesan gojek data dan gps saya akti...,tiap kali pesan gojek data dan gps saya aktif ...,tiap kali pesan gojek data dan gps saya aktif ...,"[tiap, kali, pesan, gojek, data, dan, gps, say...","[kali, pesan, gojek, data, gps, aktif, tekan, ...",kali pesan gojek data gps aktif tekan booking ...,1,positive,2,negatif
4,f2d90c51-3f25-4614-bde4-3960f50cd929,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong beri perlindungan untuk nasabah yang pe...,1,33,5.36.2,2025-10-24 10:55:00,"Hai Kak Putu, mohon maaf atas ketidaknyamanann...",2025-10-24 11:28:51,...,tolong beri perlindungan untuk nasabah yang pe...,tolong beri lindung untuk nasabah yang pesan g...,mohon beri lindung untuk nasabah yang pesan tr...,"[mohon, beri, lindung, untuk, nasabah, yang, p...","[mohon, lindung, nasabah, pesan, transportasi,...",mohon lindung nasabah pesan transportasi trans...,0,neutral,1,negatif


Handling class imbalance

In [74]:
from sklearn.utils import resample

# Hitung jumlah minimal dari tiap kelas
min_count = df['polarity'].value_counts().min()
print(df['polarity'].value_counts())

# Undersampling per kelas
df_balanced = (
    df.groupby('polarity', group_keys=False)
      .apply(lambda x: x.sample(min_count, random_state=42))
)

print(df_balanced['polarity'].value_counts())

polarity
neutral     21940
negative     8640
positive     5202
Name: count, dtype: int64
polarity
negative    5202
neutral     5202
positive    5202
Name: count, dtype: int64


/tmp/ipython-input-2489822909.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(min_count, random_state=42))


Pelatihan Model

Model 1 - TF-IDF + Random Forest (80/20)

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode label
le = LabelEncoder()
df_balanced['label'] = le.fit_transform(df_balanced['polarity'])

# Mapping label
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label mapping:", label_mapping)

Label mapping: {'negative': np.int64(0), 'neutral': np.int64(1), 'positive': np.int64(2)}


In [76]:
# Split data
X = df_balanced['text_akhir']
y = df_balanced['label']

# Split 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Data training: {len(X_train)}")
print(f"Data testing: {len(X_test)}")

Data training: 12484
Data testing: 3122


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=1000, min_df=2, max_df=0.9, ngram_range=(1,3))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df

,ad,admin,adu,ah,ajar,aju,akal,akses,aktif,aktifin,...,yg ambil,yg aplikasi,yg cancel,yg masuk,yg murah,yg pakai,yg pesan,yg suka,yg terima,yg yg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.180359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Membuat objek model RF
rf_model = RandomForestClassifier(
    n_estimators=1000,
    max_depth=20,
    random_state=42,
)

# Melatih model
rf_model.fit(X_train_tfidf, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = rf_model.predict(X_train_tfidf)
y_pred_test_rf = rf_model.predict(X_test_tfidf)

# Evaluasi akurasi model RF
accuracy_train_rf = accuracy_score(y_train, y_pred_train_rf)
accuracy_test_rf = accuracy_score(y_test, y_pred_test_rf)

# Menampilkan hasil
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9855014418455623
Random Forest - accuracy_test: 0.9590006406149904


Model 2 - TF-IDF + SVM (70/30)

In [79]:
# Split ulang
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Ekstraksi fitur TF-IDF
tfidf_2 = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf_2 = tfidf_2.fit_transform(X_train_2)
X_test_tfidf_2 = tfidf_2.transform(X_test_2)

In [80]:
from sklearn.svm import SVC

# Model SVM
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_tfidf_2, y_train_2)

# Prediksi
y_train_pred_2 = svm_model.predict(X_train_tfidf_2)
y_test_pred_2 = svm_model.predict(X_test_tfidf_2)

# Evaluasi
train_acc_2 = accuracy_score(y_train_2, y_train_pred_2)
test_acc_2 = accuracy_score(y_test_2, y_test_pred_2)

print(f"SVM - accuracy_training: {train_acc_2}")
print(f"SVM - accuracy_testing: {test_acc_2}")

SVM - accuracy_training: 0.988831929696082
SVM - accuracy_testing: 0.971806920119607


Model 3 - TF-IDF + Logistic Regression (70/30)

In [86]:
# Split ulang
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Ekstraksi fitur TF-IDF
tfidf_3 = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf_3 = tfidf_3.fit_transform(X_train_3)
X_test_tfidf_3 = tfidf_3.transform(X_test_3)

In [88]:
from sklearn.linear_model import LogisticRegression

# Membuat objek model Logistic Regression
logistic_regression = LogisticRegression()

# Latih model pakai hasil TF-IDF, bukan teks mentah
logistic_regression.fit(X_train_tfidf_3, y_train_3)

# Prediksi
y_pred_train_lr = logistic_regression.predict(X_train_tfidf_3)
y_pred_test_lr = logistic_regression.predict(X_test_tfidf_3)

# Evaluasi akurasi
accuracy_train_lr = accuracy_score(y_train_3, y_pred_train_lr)
accuracy_test_lr = accuracy_score(y_test_3, y_pred_test_lr)

print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

Logistic Regression - accuracy_train: 0.9756499450750641
Logistic Regression - accuracy_test: 0.9545066211020932
